# OpenCV Library

Basic image processing.


Image processing and computer vision tasks include displaying, cropping, flipping, rotating,  image segmentation, classification, image restoration,  image recognition, image generation.  

<ul>
    <li>In this notebook:
        <ul>
            <li>Importing and getting information about images   </li>
            <li>Histograms, Intensity Transformations  </li>
            <li>Geometric Transformations</li>
            <li>Spatial Filtering </li>
        </ul>
    </li>
    
</ul>


----


Images used in the notebook:


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/lenna.png -O lenna.png
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/baboon.png -O baboon.png
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/barbara.png -O barbara.png  
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/goldhill.bmp -O goldhill.bmp
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/cameraman.jpeg -O cameraman.jpeg
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/zelda.png -O zelda.png
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-CV0101EN-SkillsNetwork/images%20/images_part_1/mammogram.png -O mammogram.png

Helper function to concatenate two images side-by-side. 

In [2]:
def get_concat_h(im1, im2):
    #https://note.nkmk.me/en/python-pillow-concat-images/
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

In [3]:
my_image = "lenna.png"

In [4]:
# checking current path
import os
cwd = os.getcwd()
cwd 

'c:\\Users\\Alberto\\Desktop\\github\\deep-learning-methods'

In [5]:
# checking path of the image
image_path = os.path.join(cwd, my_image)
image_path


'c:\\Users\\Alberto\\Desktop\\github\\deep-learning-methods\\lenna.png'

### `Importing and getting information about images`


In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

The <code>imread()</code> method loads an image from the specified file, the input is the <code>path</code> of the image to be read, the <code>flag</code> paramter specifies how the image should be read, and the default value is <code>cv2.IMREAD_COLOR</code>.


In [7]:
image = cv2.imread(my_image)

# to load the image using the path
# image = cv2.imread(image_path)
# image.shape

The result is a numpy array with intensity values as 8-bit unsigned integers. 


In [8]:
type(image)

numpy.ndarray

Shape of the array from the `shape` attribute.


In [9]:
image.shape

(512, 512, 3)

Other libraries such as PIL return images in (R, G, B) format whereas OpenCV returns in (B, G, R) format.


In [10]:
image.max()

255

and


In [11]:
image.min()

3

OpenCV's `imshow` function can open images in a new window, but this may bring problems in Jupyter:


In [ ]:
#cv2.imshow('image', imgage)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

Using the `imshow` function from the `matplotlib` library:


In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.show()

Changing the color space with conversion code and the function `cvtColor` from the `cv2` library:


In [ ]:
new_image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(new_image)
plt.show()

Grayscale images have pixel values representing the amount of light or intensity. Light shades of gray have a high-intensity darker shades have a lower intensity. White has the highest intensity, and black the lowest. 

To convert an image to Gray Scale using a color conversion code there is the function <code>cvtColor</code> and the parameter <code>cv2.COLOR_BGR2GRAY</code>.


In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

The image array has only two dimensions, i.e. only one color channel:


In [ ]:
image_gray.shape

In [ ]:
# to save the grayscale image
cv2.imwrite('lena_gray_cv.jpg', image_gray)

To load a grayscale image `cv2.imread()` needs the gray color conversation code <code>cv2.COLOR_BGR2GRAY</code>.


In [ ]:
im_gray = cv2.imread('barbara.png', cv2.IMREAD_GRAYSCALE)

Working with different color channels.


In [ ]:
baboon=cv2.imread('baboon.png')
plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(baboon, cv2.COLOR_BGR2RGB))
plt.show()

Getting the separate RGB channels and assigning them to the variables <code>blue</code>, <code>green</code>, and <code>red</code>, in (B, G, R) format.


In [ ]:
blue, green, red = baboon[:, :, 0], baboon[:, :, 1], baboon[:, :, 2]

Concatenating each image channel using the function <code>vconcat</code>.


In [ ]:
im_bgr = cv2.vconcat([blue, green, red])

Plotting the color image next to the red channel in grayscale.
Regions with red have higher intensity values.


In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(cv2.cvtColor(baboon, cv2.COLOR_BGR2RGB))
plt.title("RGB image")
plt.subplot(122)
plt.imshow(im_bgr,cmap='gray')
plt.title("Different color channels  blue (top), green (middle), red (bottom)  ")
plt.show()

We can use numpy slicing to return the first 256 rows corresponding to the top half of the image:


In [ ]:
rows = 256

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(new_image[0:rows,:,:])
plt.show()

Or to the first 256 columns corresponding to the left half of the image:


In [ ]:
columns = 256

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(new_image[:,0:columns,:])
plt.show()

To reassign an array to another variable, use the `copy` method.

In [ ]:
A = new_image.copy()
plt.imshow(A)
plt.show()

If the method `copy()` is not applyied, the variable will point to the same location in memory.


Manipulating elements by index. Below, the red channel of the new array `baboon_red` is set to zero making the image look red.


In [ ]:
baboon_red = baboon.copy()
baboon_red[:, :, 0] = 0
baboon_red[:, :, 1] = 0
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(baboon_red, cv2.COLOR_BGR2RGB))
plt.show()

### `Histograms, Intensity Transformations`  


- Pixel Transformations as operations performed in one pixel at a time. 
- Histograms to display the distribution of intensities in an image. It can be used to optimize image characteristics. 
- Intensity Transformations to make objects easier to see by improving image contrast and brightness. 
- Thresholding to segment objects in images.

In [ ]:
# Helper function to plot two images side-by-side.
def plot_image(image_1, image_2,title_1="Orignal", title_2="New Image"):
    plt.figure(figsize=(10,10))
    plt.subplot(1, 2, 1)
    plt.imshow(image_1,cmap="gray")
    plt.title(title_1)
    plt.subplot(1, 2, 2)
    plt.imshow(image_2,cmap="gray")
    plt.title(title_2)
    plt.show()

# Helper function to plot two histograms side-by-side
def plot_hist(old_image, new_image,title_old="Orignal", title_new="New Image"):
    intensity_values=np.array([x for x in range(256)])
    plt.subplot(1, 2, 1)
    plt.bar(intensity_values, cv2.calcHist([old_image],[0],None,[256],[0,256])[:,0],width = 5)
    plt.title(title_old)
    plt.xlabel('intensity')
    plt.subplot(1, 2, 2)
    plt.bar(intensity_values, cv2.calcHist([new_image],[0],None,[256],[0,256])[:,0],width = 5)
    plt.title(title_new)
    plt.xlabel('intensity')
    plt.show()

A histogram counts the number of occurrences per pixel for each intensity value. It's a useful tool for understanding and manipulating images.

<code>cv.calcHist()</code> is used to generate a histogram. 

Here are the parameter values:
<p>
    <code>cv2.calcHist(
                        CV array:<b>[image]</b>,
                        image channel:<b>[0]</b>,
                        <b>[None]</b>,
                        number of bins:<b>[L]</b>,
                        index range of bins:<b>[0,L-1]</b>) </code>   
</p>
For real images, <code>L</code> is <code>256</code>.

Histogram of a toy array with intensity values ranging from 0 to 2. Its first element is the number of zeros in the image (in this case, 1); its second element is the number of ones in the image (in this case, 5).

In [ ]:
toy_image = np.array([[0,2,2],[1,1,1],[1,1,2]],dtype=np.uint8)
plt.imshow(toy_image, cmap="gray")
plt.show()
print("toy_image:",toy_image)

The <code>caclHist</code> function calculates the histogram, in this case, there are only three bins for the three values. The index of the bins range from 1 to 3.

In [ ]:
plt.bar([x for x in range(6)],[1,5,2,0,0,0])
plt.show()

### `Geometric Transformations`


**Geometric Operations:**

- Scaling
- Translation
- Rotation


**Mathematical Operations:**

- Array Operations
- Matix Operations


### `Spatial Filtering` 

- Filtering Noise
- Gaussian Blur
- Image Sharpening
- Edges
- Median
- Threshold